# softmax回归的从零开始实现

softmax 回归也是重要的基础，因此你应该知道实现 softmax 回归的细节。还是使用 Fashion-mnist 数据集


In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

## 初始化模型参数

目前我们认为 28*28 图像展平为一维，每个像素都为一个特征，一张图片共有 784 个特征。后面在学习如何处理多维图像的特征

由于有十个种类，所以网络设置为具有 10 个输出。所以当网络只有两层的时候，权值矩阵的尺寸为 $(784, 10)$，偏置也组成一个 10 维向量


In [2]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

## 定义 Softmax 操作

In [5]:
def Softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition # 这里应用了广播机制

## 定义模型

In [6]:
def net(X):
    return Softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

## 损失函数

一个问题吧，例如我们用的 Fashion-mnist 数据集，它的标注只有一个数字，例如 3 代表这是第四类、0 代表这是第一类。但是，我们经过网络计算的出来的值是有 10 个输出的。怎么把标注这一个数组变成类似于 `[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]` 这样的代表一类正确的形式然后再计算损失函数，然后计算出每个输出上的 loss 作为一个向量呢？

$$
l(\hat{y}, y) = - \sum_{j=1}^{q} y_{j} \log{\hat{y_{j}}}
$$

我们的方法是这样，每次看都仔细读读吧！

（[这里](https://zh-v2.d2l.ai/chapter_linear-networks/softmax-regression-scratch.html#id4)有一个引论）

In [7]:
def Cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])